## <b><u> Time series Forecasting for Favorita Store Sales </b></u>
   - `In this project we will be building a regression model and understand hw to use it`
   - `We will also examine the correlation coeffcient,multiple regression  and logisti regression methods`
   - `using logistic regression we will be able to estimate the probability of the business success over give period of time`
   - `finally; develop a time series Analysis`



### <u><b><i> Description </u></b></i>
` As a data scientist, I will be looking at a favorita  grocery stores data  and predict a little over time based on th sales performance previously... </ `
 - The Favorita Grocery Sales Forecasting competition is a time series forecasting competition hosted by the Favorita Food Markets Association. The goal is to predict the sales of various products sold in multiple locations over time. The dataset provided includes sales data, promotions, and other relevant features.
 - `The dataset includes;;;;;;;;;;;;`
- train.csv: includes store_number, family, onpromotion and target sales
- test.csv: same as train.csv(predict sales data)
- transaction.csv: date,store_number,transation 
- sample_submission.csv: 
- stores.csv: store metabase including city, state,type and cluster
 - oils.csv: daily oil price
 - holidays_events.csv: Holidays events
 ## Questions
 1. Is the train dataset complete (has all the required dates)?
 2. Which dates have the lowest and highest sales for each year (excluding days the store was closed)
 3. Compare the sales for each month across the years and determine which month of which year had the highest sales. 
 4. Did the earthquake impact sales?
 5. Are certain stores or groups of stores selling more products? (Cluster, city, state, type)
 6. Are sales affected by promotions, oil prices and holidays
 7. What analysis can we get from the date and its extractable features?
 8. Which product family and stores did the promotions affect.
 9. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)
 10. Does the payment of wages in the public sector on the 15th and last days of the month influence the store sales.
 ### <u><b><i> HYPOTHESIS</u></b></i>
Null hypothesis;`The type of holiday does not have any significance on the increase of sales`
Alternative hypothesis; `The type of holiday do have significant on the increase of sales`

In [1]:
#installing pyodbc & python-dotenv
import pyodbc
from dotenv import dotenv_values
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
#load environment variable from .env file
environment_variable =dotenv_values('.env')
#we get the credentails  fro th env file
database=environment_variable.get('DATABASE')
server=environment_variable.get('SERVER')
username=environment_variable.get('USERNAME')
password=environment_variable.get('PASSWORD')
# Establish connection
connection_string= f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"

In [3]:
#We need to use the connection method that is the pyodbc library
connection=pyodbc.connect(connection_string)

In [4]:
#SQL query to get the dat 
query_oil="SELECT* FROM dbo.oil"
query_stores="SELECT* FROM dbo.stores"
query_holidays_events="SELECT* FROM dbo.holidays_events"
df_oil=pd.read_sql(query_oil,connection)
df_stores=pd.read_sql(query_stores,connection)
df_holidays_events=pd.read_sql(query_holidays_events,connection)
# Close the connection





In [6]:
#load csv data set
df_test=pd.read_csv("test.csv")
df_transactions=pd.read_csv("transactions.csv")
df_sample_submission=pd.read_csv("sample_submission.csv")




In [7]:
##sql EDA
#summary statisitcs for th column
oil_sum=df_oil.describe().T
event_sum=df_holidays_events.describe().T
store_sum=df_stores.describe().T
##result
print("holiday summary:")
display(event_sum)
print('*'*40)
print("Store Summary:")
display(store_sum)
print('$'*40)
print("oil summary:")
display(oil_sum)
print('-'*40)




holiday summary:


,count,unique,top,freq
date,350,312,2014-06-25,4
type,350,6,Holiday,221
locale,350,3,National,174
locale_name,350,24,Ecuador,174
description,350,103,Carnaval,10
transferred,350,2,False,338


****************************************
Store Summary:


,count,mean,std,min,25%,50%,75%,max
store_nbr,54.0,27.500000,15.732133,1.0,14.25,27.5,40.75,54.0
cluster,54.0,8.481481,4.693395,1.0,4.00,8.5,13.00,17.0


$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
oil summary:


,count,mean,std,min,25%,50%,75%,max
dcoilwtico,1175.0,67.714366,25.630476,26.190001,46.405001,53.189999,95.66,110.620003


----------------------------------------


In [8]:
#missing values
print("event missing values:")
events_missing=df_holidays_events.isnull().sum()
display(events_missing)
print('*'*40)
print("oil missing values:")
oil_missing=df_oil.isnull().sum()
display(oil_missing)
print('*'*40)
print("store missing values:")
store_missing=df_stores.isnull().sum()
display(store_missing)
print('*'*40)

event missing values:


date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

****************************************
oil missing values:


date           0
dcoilwtico    43
dtype: int64

****************************************
store missing values:


store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

****************************************


In [9]:
print("holiday shape:")
print('*'*40)
holiday_shape=df_holidays_events.shape
print("rows:",holiday_shape[0])
print("column",holiday_shape[1])
#oilshape
print("oil shape:")
print('*'*40)
oil_shape=df_oil.shape
print("rows:",oil_shape[0])
print("column",oil_shape[1])

#store shape
print("store shape:")
print("<>"*40)
store_shape=df_stores.shape
print("rows:",store_shape[0])
print("column:",store_shape[1])



holiday shape:
****************************************
rows: 350
column 6
oil shape:
****************************************
rows: 1218
column 2
store shape:
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
rows: 54
column: 5
